In [2]:
import pandas as pd

# users = pd.read_csv('data/users_export.csv')
ratings = pd.read_csv('data/ratings_export.csv')
movies = pd.read_csv('data/movie_data.csv', on_bad_lines='skip')
my_ratings = pd.read_csv('data/my_ratings.csv')

# Merge datasets to create a comprehensive dataset
user_ratings = pd.merge(ratings, movies, left_on='movie_id', right_on='movie_id')

In [3]:
user_ratings.head()

,_id_x,movie_id,rating_val,user_id,_id_y,genres,image_url,imdb_id,imdb_link,movie_title,...,popularity,production_countries,release_date,runtime,spoken_languages,tmdb_id,tmdb_link,vote_average,vote_count,year_released
0,5fc57c5d6758f6963451a07f,feast-2014,7,deathproof,5fc880726758f69634df0bca,"[""Animation"",""Comedy"",""Drama"",""Family""]",film-poster/2/2/0/1/9/2/220192-feast-0-230-0-3...,tt3689498,http://www.imdb.com/title/tt3689498/maindetails,Feast,...,9.26,"[""United States of America""]",2014-10-25,6.0,"[""English""]",293299.0,https://www.themoviedb.org/movie/293299/,7.9,720.0,2014.0
1,5fc57ca06758f69634538bad,feast-2014,10,ingridgoeswest,5fc880726758f69634df0bca,"[""Animation"",""Comedy"",""Drama"",""Family""]",film-poster/2/2/0/1/9/2/220192-feast-0-230-0-3...,tt3689498,http://www.imdb.com/title/tt3689498/maindetails,Feast,...,9.26,"[""United States of America""]",2014-10-25,6.0,"[""English""]",293299.0,https://www.themoviedb.org/movie/293299/,7.9,720.0,2014.0
2,5fc57cbd6758f696345475a3,feast-2014,8,dirkh,5fc880726758f69634df0bca,"[""Animation"",""Comedy"",""Drama"",""Family""]",film-poster/2/2/0/1/9/2/220192-feast-0-230-0-3...,tt3689498,http://www.imdb.com/title/tt3689498/maindetails,Feast,...,9.26,"[""United States of America""]",2014-10-25,6.0,"[""English""]",293299.0,https://www.themoviedb.org/movie/293299/,7.9,720.0,2014.0
3,5fc57ce06758f6963455400b,feast-2014,10,childrenofmen,5fc880726758f69634df0bca,"[""Animation"",""Comedy"",""Drama"",""Family""]",film-poster/2/2/0/1/9/2/220192-feast-0-230-0-3...,tt3689498,http://www.imdb.com/title/tt3689498/maindetails,Feast,...,9.26,"[""United States of America""]",2014-10-25,6.0,"[""English""]",293299.0,https://www.themoviedb.org/movie/293299/,7.9,720.0,2014.0
4,5fc57cf36758f69634558b0e,feast-2014,8,suspirliam,5fc880726758f69634df0bca,"[""Animation"",""Comedy"",""Drama"",""Family""]",film-poster/2/2/0/1/9/2/220192-feast-0-230-0-3...,tt3689498,http://www.imdb.com/title/tt3689498/maindetails,Feast,...,9.26,"[""United States of America""]",2014-10-25,6.0,"[""English""]",293299.0,https://www.themoviedb.org/movie/293299/,7.9,720.0,2014.0


In [4]:
len(user_ratings)

11079666

In [5]:
len(my_ratings)

85

In [6]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.metrics.pairwise import cosine_similarity

# Normalize movie titles for better matching
movies['movie_title'] = movies['movie_title'].str.lower().str.strip()
my_ratings['Name'] = my_ratings['Name'].str.lower().str.strip()

# Merge your ratings with the movies DataFrame to get 'movie_id' and directly assign 'user_id' = 0
merged = my_ratings.merge(movies[['movie_id', 'movie_title']], left_on='Name', right_on='movie_title', how='inner')
my_ratings_updated = merged[['movie_id', 'Rating']].copy()  # Use .copy() to explicitly create a copy
my_ratings_updated.loc[:, 'user_id'] = 0  # Use .loc to avoid SettingWithCopyWarning

my_ratings_updated

,movie_id,Rating,user_id
0,interstellar,5.0,0
1,500-days-of-summer,4.5,0
2,crazy-stupid-love,3.5,0
3,friends-with-benefits,3.5,0
4,the-hangover,3.0,0
...,...,...,...
142,the-princess-bride,4.0,0
143,ghosted-2011,2.5,0
144,ghosted,2.5,0
145,ghosted-2020,2.5,0


In [7]:

# Add your ratings to the user ratings DataFrame
combined_ratings = pd.concat([user_ratings, my_ratings_updated.rename(columns={'Rating': 'rating_val'})])

# Map movie IDs and user IDs to indices for creating a sparse matrix
movie_id_to_idx = {movie_id: i for i, movie_id in enumerate(combined_ratings['movie_id'].unique())}
user_id_to_idx = {user_id: i for i, user_id in enumerate(combined_ratings['user_id'].unique())}

# Create a sparse matrix of ratings
rows = combined_ratings['user_id'].map(user_id_to_idx)
cols = combined_ratings['movie_id'].map(movie_id_to_idx)
data = combined_ratings['rating_val']
ratings_matrix = csr_matrix((data, (rows, cols)), shape=(len(user_id_to_idx), len(movie_id_to_idx)))

# Compute cosine similarity between users
user_similarity = cosine_similarity(ratings_matrix)
user_similarity

array([[1.        , 0.49803018, 0.34356284, ..., 0.01531783, 0.02144497,
        0.09140448],
       [0.49803018, 1.        , 0.36897997, ..., 0.        , 0.        ,
        0.11030959],
       [0.34356284, 0.36897997, 1.        , ..., 0.        , 0.        ,
        0.0937588 ],
       ...,
       [0.01531783, 0.        , 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.02144497, 0.        , 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.09140448, 0.11030959, 0.0937588 , ..., 0.        , 0.        ,
        1.        ]])

In [8]:
# Since you are user_id 0, find the most similar users to you
top_100_similar_users_indices = np.argsort(-user_similarity[0])[1:101]  # Get indices of top 100 similar users

my_rated_movies = set(my_ratings_updated['movie_id'])

# Initialize a dictionary to hold recommended movies and the users who recommended them
recommended_movies_details = {}

for user_index in top_100_similar_users_indices:
    user_id = list(user_id_to_idx.keys())[list(user_id_to_idx.values()).index(user_index)]
    
    # # Filter movies rated 5/5 by the user
    # high_rated_movies_by_user = combined_ratings[(combined_ratings['user_id'] == user_id) & (combined_ratings['rating_val'] == 5)]
    high_rated_movies_by_user = combined_ratings[(combined_ratings['user_id'] == user_id)]
    
    for _, row in high_rated_movies_by_user.iterrows():
        movie_id = row['movie_id']
        if movie_id not in my_rated_movies:
            if movie_id not in recommended_movies_details:
                recommended_movies_details[movie_id] = {'users': [user_id], 'ratings': [row['rating_val']]}
            else:
                recommended_movies_details[movie_id]['users'].append(user_id)
                recommended_movies_details[movie_id]['ratings'].append(row['rating_val'])

# Limit to top 5 based on the number of users recommending the movie
recommended_movies_ids = sorted(recommended_movies_details, key=lambda x: len(recommended_movies_details[x]['users']), reverse=True)[:10]

# Fetch movie titles and stats
recommended_titles_and_stats = []
for movie_id in recommended_movies_ids:
    movie_title = movies[movies['movie_id'] == movie_id]['movie_title'].iloc[0]
    avg_rating = np.mean(recommended_movies_details[movie_id]['ratings'])
    num_users = len(recommended_movies_details[movie_id]['users'])
    recommended_titles_and_stats.append({
        'title': movie_title,
        'average_rating': avg_rating,
        'recommended_by_users_count': num_users,
        'recommended_by_user_ids': recommended_movies_details[movie_id]['users']
    })

# Display recommended movies along with stats
for movie in recommended_titles_and_stats:
    print(f"Title: {movie['title']}, Avg Rating: {movie['average_rating']:.2f}, Recommended by {movie['recommended_by_users_count']} Users, User IDs: {movie['recommended_by_user_ids']}")

Title: the social network, Avg Rating: 9.38, Recommended by 100 Users, User IDs: ['adamnotsandler', 'bartonfinked', 'childrenofmen', 'alicjas', 'kevschwa', 'sushifern', 'joshuatbrown', 'kwang', 'lukelivermore', 'ianamurray', 'phntmthrd', 'hotdad23', 'juliendonkeyboi', 'mrmichaeldjones', 'jessiebuckley', 'hauteprincesses', 'hewasthrfriendd', 'lesaladino', 'ingridgoeswest', 'samkohn', 'haber345', 'tburnett217', 'sadiemuriel', 'livmallia', 'akroeker', 'zaneg4l', 'tschutzer', 'dazapata', 'babydriverreal', 'eleanordempsey', 'vprento', 'hotblonde', 'zyopy', 'amandaayoung', 'potato_digest', 'thecinemalover', 'sonofjorel', 'samslabyrinth', 'ezracubero', 'tydedermody', 'mknightshymalan', 'rjrucker', 'sloanepeterson', 'flaska11', 'thekingslayer', 'ashmatxx', 'hollowbody', 'suspirium', 'bencaplan', 'heyitsvasili', 'bratpitt', 'mouserat', 'leaseydouxx', 'whittystuff', 'bkkirby', 'lishsloans', 'bbanks2121', 'tracesauveur', 'mank', 'tommysyk', 'unofficial_rt', 'prashprash', 'gainsbourg', 'moxleyox',

In [9]:
# output to file

with open('data/recommended_movies.txt', 'w') as f:
    for movie in recommended_titles_and_stats:
        f.write(f"Title: {movie['title']}, Avg Rating: {movie['average_rating']:.2f}, Recommended by {movie['recommended_by_users_count']} Users, User IDs: {movie['recommended_by_user_ids']}\n")